In [1]:
import pandas as pd
import plotly.express as px

In [2]:
sm_a = pd.read_csv('./a/records/sm')

batt_a = pd.read_csv('./a/data/battery')
itemp_a = pd.read_csv('./a/data/itemp')


sm_c = pd.read_csv('./c/records/sm')

batt_c = pd.read_csv('./c/data/battery')
itemp_c = pd.read_csv('./c/data/itemp')

sm_a.columns = sm_a.columns.str.strip()
batt_a.columns = batt_a.columns.str.strip()
itemp_a.columns = itemp_a.columns.str.strip()

sm_c.columns = sm_c.columns.str.strip()
batt_c.columns = batt_c.columns.str.strip()
itemp_c.columns = itemp_c.columns.str.strip()

In [3]:
sm_a_filtered = sm_a[4:].drop(columns=['SM3']).copy()

batt_a_filtered = batt_a[4:].copy()
itemp_a_filtered = itemp_a[4:].copy()

sm_c_filtered = sm_c[1:].copy()

batt_c_filtered = batt_c[1:].copy()
itemp_c_filtered = itemp_c[1:].copy()

In [4]:
def convert_to_sm(raw_value, raw_dry, raw_sat):
    sm_value = (raw_value - raw_dry) / (raw_sat - raw_dry)
    return sm_value

station_a = sm_a_filtered.copy()
station_a['batt_v'] = batt_a_filtered['Battery Voltage']
station_a['batt_p'] = batt_a_filtered['Battery Percentage']
station_a['itemp'] = itemp_a_filtered['Internal Temperature']

station_a[['SM1']] = sm_a_filtered[['SM1']].map(lambda x: convert_to_sm(x, raw_dry=21_000, raw_sat=8_000))
station_a[['SM2']] = sm_a_filtered[['SM2']].map(lambda x: convert_to_sm(x, raw_dry=21_000, raw_sat=8_000))

station_a['batt_v'] = station_a['batt_v'] * 2.3
min_voltage, max_voltage = 3.2, 4.2
station_a['batt_p'] = ((station_a['batt_v'] - min_voltage) / (max_voltage - min_voltage)) * 100

station_a['itemp'] = station_a['itemp'] + 273 + 25



station_c = sm_c_filtered.copy()
station_c['batt_v'] = batt_c_filtered['Battery Voltage']
station_c['batt_p'] = batt_c_filtered['Battery Percentage']
station_c['itemp'] = itemp_c_filtered['Internal Temperature']

station_c[['SM1']] = sm_c_filtered[['SM1']].map(lambda x: convert_to_sm(x, raw_dry=21_000, raw_sat=8_000))
station_c[['SM2']] = sm_c_filtered[['SM2']].map(lambda x: convert_to_sm(x, raw_dry=21_000, raw_sat=8_000))
station_c[['SM3']] = sm_c_filtered[['SM3']].map(lambda x: convert_to_sm(x, raw_dry=21_000, raw_sat=8_000))

station_c['batt_v'] = station_c['batt_v'] * 1.9
min_voltage, max_voltage = 3.2, 4.2
station_c['batt_p'] = ((station_c['batt_v'] - min_voltage) / (max_voltage - min_voltage)) * 100

station_c['itemp'] = station_c['itemp'] + 273 + 25 + 130

In [5]:
station_a_plot_sm = station_a[['Time', 'SM1', 'SM2']].melt(id_vars=['Time'], value_name='SM', var_name='SMID')

station_c_plot_sm = station_c[['Time', 'SM1', 'SM2', 'SM3']].melt(id_vars=['Time'], value_name='SM', var_name='SMID')

In [6]:
px.line(station_a_plot_sm, x='Time', y='SM', color='SMID', title='Station A: Soil Moisture (VWC eqv., uncalibrated)').show()

In [7]:
px.line(station_c_plot_sm, x='Time', y='SM', color='SMID', title='Station C Soil Moisture (VWC eqv., uncalibrated)').show()

In [8]:
px.line(station_a, x='Time', y='batt_p', title='Station A: Battery Percentage').show()

In [9]:
px.line(station_c, x='Time', y='batt_p', title='Station C: Battery Percentage').show()

In [10]:
px.line(station_a, x='Time', y='itemp', title='Station A: Internal Temperature').show()

In [11]:
px.line(station_c, x='Time', y='itemp', title='Station C: Internal Temperature').show()